In [1]:
# OpenAI API Key 정의
# **주의!** OpenAI API Key 는 외부에 노출하지 않도록 주의한다. 
# 4월 17일 이후에는 이전에 사용하던 사용자 키(User Key)보다 프로젝트별로 구분할 수 있는 프로젝트 키(Project Key)를 활용하는 것을 권장
# 일반적으로 시스템의 환경변수에 숨겨놓고 소스코드에는 노출하지 않지만, 
# 해당 과정은 파이썬 활용에 관한 내용이므로 예제에서는 생략한다.
# [모델 문서](https://platform.openai.com/docs/models)에는 사용가능한 모델의 이름이 나열되어있고, OpenAI에서 모델을 업데이트할 때마다 이곳에 명시하므로 자주 확인하면 좋다.

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print(f"> 구글 코랩의 보안 비밀에서 OPENAI_API_KEY를 불러옵니다.")

except:
    OPENAI_API_KEY = "YOUR_OPENAI_PROJECT_KEY"
    print(f"> 로컬 환경에서 설정한 OPENAI_API_KEY를 사용합니다.")

OPENAI_MODEL = "gpt-4-turbo"

In [2]:
# OpenAI API 패키지 import
try:
    import openai

except ModuleNotFoundError:
    # OpenAI API 패키지 설치
    !pip install openai

finally:
    import openai

if "1.23.1" != openai.VERSION:
    raise "예제는 2024-04-17 update 버전을 기준으로 작성하였습니다."

### [Assistants Tool] Code Interpreter

어시스턴트가 스스로 파이썬 코드를 작성하고 실행

In [5]:
from pathlib import Path
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 0. Assistant 용도로 파일을 OpenAI 서버의 프로젝트 저장소에 파일 업로드
# 데이터 출처: [국토교통부 실거래가 공개시스템](https://rtmobile.molit.go.kr/) 아파트(전월세) 실거래가
file = client.files.create(
    file=open("datafiles/아파트(전월세)_실거래가_2024년1월.csv", "rb"),
    purpose='assistants',
)

# 1. 스레드 생성 
thread = client.beta.threads.create()

# 2. 어시스턴트 생성; 코드 인터프리터 도구 및 참조 데이터 파일 지정하고 역할 부여
assistant = client.beta.assistants.create(
    name="데이터 분석가",
    instructions="""
        당신은 데이터 분석가입니다. 
        질문에 대한 데이터를 주어진 파일에서 찾아서 분석 코드를 작성하고 실행합니다.
        그래프를 그리는 경우, 그래프 구성요소 중 한글은 영문으로 변환하고, png 이미지 파일로 제공합니다.
        """,
    model="gpt-4-turbo",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id]
        }
    }
)

# 3. 사용자 메시지 생성; 메시지의 내용에 그래프를 그리고, 
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="서울 특별시의 마포구, 용산구, 성동구, 노원구의 전세 계약 보증금을 나타내는 바이올린 플롯 (차트 범례와 구 명은 영어로 표기)"
)

print(f"{message.role} >")
print(message.content[0].text.value)

# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 런 스탭 내용 확인
if run.status == 'completed':
    # 런에 대한 런 스탭 목록 가져오기      
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id,
        order="asc" # 런 스탭을 시간순으로 가져오기
    )   
    
    # 각 스텝에대한 소스코드 코드 로그 또는 메시지 표시 혹은 메시지의 주석 파일 다운로드
    for steps in run_steps.data:
        if "tool_calls" == steps.type:
            for tool_call in steps.step_details.tool_calls:
                if "code_interpreter" == tool_call.type:
                    print("\n```python")
                    print(tool_call.code_interpreter.input)
                    for output in tool_call.code_interpreter.outputs:
                        if "log" == output.type:
                            print(output.logs)
                    print("```\n")

        elif "message_creation" == steps.type:
            if "completed" == steps.status:
                msgid = steps.step_details.message_creation.message_id
                message = client.beta.threads.messages.retrieve(
                    thread_id=thread.id,
                    message_id=msgid
                )
                print(f"{message.role} >")
                for content in message.content:
                    if "text" == content.type:
                        print(content.text.value)
                        
                        # 주석 파일을 저장
                        for annotation in content.text.annotations:
                            if "file_path" == annotation.type:
                                image_data_bytes = client.files.content(
                                    file_id=annotation.file_path.file_id
                                ).read()
                                
                                filename = Path(annotation.text).name
                                                              
                                with open(f"image/{filename}", "wb") as fp:
                                    fp.write(image_data_bytes)                       
                                
else:
    print(run.status)

# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")

# 8. 파일 삭제
res = client.files.delete(file_id=file.id)
print(f"file deleted: {res.deleted}")


user >
서울 특별시의 마포구, 용산구, 성동구, 노원구의 전세 계약 보증금을 나타내는 바이올린 플롯 (차트 범례와 구 명은 영어로 표기)
assistant >
먼저 데이터 파일의 내용과 구조를 확인하여 필요한 데이터를 추출해야 합니다. 파일 형식 및 데이터를 확인하겠습니다.

```python
import pandas as pd

# 파일 경로
file_path = '/mnt/data/file-QiYvpLxkQ4wluKVMyNEYInG7'

# 파일 형식 추측하기 위해 헤더만 먼저 읽어보기
try:
    # 가정 CSV 형식
    data_head = pd.read_csv(file_path, nrows=5)
except Exception as e:
    print(e)

data_head
```

assistant >
처음 시도한 파일 읽기에서 표준 데이터 형식이 아닌 것 같습니다. 후속 행에서 데이터의 구조를 파악하고 올바르게 읽을 수 있는 방법을 찾기 위해 파일의 첫 10줄을 직접 출력해 보겠습니다.

```python
# 파일의 처음 10줄을 확인하기 위해 직접 읽어 출력
with open(file_path, 'r', encoding='utf-8') as file:
    first_lines = [next(file) for _ in range(10)]

first_lines
```

assistant >
파일의 내용이 영문 주석 및 정보로 시작하고 있으며, 실제 데이터 형식이나 데이터의 시작 위치는 확인되지 않았습니다. 주석 다음의 데이터를 확인하기 위해 몇 줄 더 읽어보겠습니다.

```python
# 파일을 조금 더 읽어서 데이터 구조 확인
with open(file_path, 'r', encoding='utf-8') as file:
    lines = [next(file) for _ in range(30)]

lines[10:]  # 주석 이후의 데이터 확인
```

assistant >
데이터 파일의 구조를 이해하기까지 몇 줄이 필요